In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import io
import requests

In [2]:
#checking file encoding 
import chardet
def find_encoding(fname):
    r_file = open(fname, 'rb').read()
    result = chardet.detect(r_file)
    charenc = result['encoding']
    return charenc

my_encoding = find_encoding('companies.txt')

In [3]:
#Loading data to companies and round2 dataframe

companies=pd.read_csv('companies.txt',encoding=my_encoding,engine='python',sep='\t') #companies file is tab seperated file 
rounds2=pd.read_csv('rounds2.csv',encoding=my_encoding,engine='python')

In [4]:
companies.head(2)

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014


In [5]:
rounds2.head(2)

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN


### Removing Special Character 

In [6]:

rounds2.iloc[31863]['company_permalink']

'/ORGANIZATION/ENERGYSTONE-GAMES-Ç\x81ľÇ\x9fłĆ¸¸Ć\x88\x8f'

In [7]:
companies.iloc[18197]['permalink']

'/Organization/Energystone-Games-Ç\x81ľç˙łĆ¸¸Ć\x88\x8f'

In [8]:
#Removing Special Character 
companies.permalink=companies.permalink.str.encode('utf-8').str.decode('ascii', 'ignore')
rounds2.company_permalink=rounds2.company_permalink.str.encode('utf-8').str.decode('ascii', 'ignore')

In [9]:
#Validating after special character removal 
rounds2.iloc[31863]['company_permalink']

'/ORGANIZATION/ENERGYSTONE-GAMES-'

In [10]:
companies.iloc[18197]['permalink']

'/Organization/Energystone-Games-'

### Checkpoint 1: Data Cleaning 1

In [11]:
#How many unique companies are present in rounds2?
distinct_company_count_round2=rounds2.company_permalink.str.lower().nunique()
print ('Number of  companies present for round2  :- {}'.format(distinct_company_count_round2))

Number of  companies present for round2  :- 66368


In [12]:
#How many unique companies are present in the companies file?
distinct_company_count=companies.permalink.str.lower().nunique()
print ('Number of  companies present for companies dataframe   :- {}'.format(distinct_company_count))

Number of  companies present for companies dataframe   :- 66368


In [13]:
#In the companies data frame, which column can be used as the  
#unique key for each company? Write the name of the column.
companies.describe().loc[['unique','count']]

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
unique,66368,66102,61191,27296,4,137,311,1092,5111,3978
count,66368,66367,61310,63220,66368,59410,57821,58338,58340,51147


In [14]:
#Are there any companies in the rounds2 file which are not  present in companies ?

rounds2[~rounds2.company_permalink.str.lower().isin(companies.permalink.str.lower())]

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd


In [15]:
##Merge the two data frames so that all  variables (columns)  in the companies frame are added to the rounds2 
##data frame. Name the merged frame master_frame. How many observations are present in master_frame?

#Upading company_permalink and permalink to lower case
companies['permalink']=companies.permalink.str.lower()
rounds2['company_permalink']=rounds2.company_permalink.str.lower()

#Merging two dataframe on permalink.

master_frame=pd.merge(rounds2,companies,left_on='company_permalink',right_on='permalink',how='left')


In [16]:
#Validating join , checking if company_permalink is same as permalink or not 
#below code should return zeo row if join is correct 

master_frame[master_frame['permalink']!=master_frame['company_permalink']]

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at


In [17]:
#Row count for master_frame
len(master_frame.index)

114949

### Checkpoint 2: Funding Type Analysis

In [18]:
# Average funding amount of venture type
print ("Average funding amount of venture type:- {} milion USD".format(round(master_frame[master_frame.funding_round_type=='venture']['raised_amount_usd'].mean()/1000000,2)))

Average funding amount of venture type:- 11.75 milion USD


In [19]:
##Average funding amount of angel type	
print ("Average funding amount of angel type:- {} milion USD".format(round(master_frame[master_frame.funding_round_type=='angel']['raised_amount_usd'].mean()/1000000,2)))


Average funding amount of angel type:- 0.96 milion USD


In [20]:
# Average funding amount of seed type
print ("Average funding amount of seed type:- {} milion USD".format(round(master_frame[master_frame.funding_round_type=='seed']['raised_amount_usd'].mean()/1000000,2)))

Average funding amount of seed type:- 0.72 milion USD


In [21]:
#Average funding amount of private equity type
print ("Average funding amount of private equity type:- {} milion USD".format(round(master_frame[master_frame.funding_round_type=='private_equity']['raised_amount_usd'].mean()/1000000,2)))

Average funding amount of private equity type:- 73.31 milion USD


In [22]:
##Considering that Spark Funds wants to invest between 5 to 15 million USD per investment round,
##which investment type is the most suitable for it?
investment_milion_usd=master_frame.groupby('funding_round_type').mean()/1000000
investment_milion_usd[investment_milion_usd.raised_amount_usd.between(5,15)]

,raised_amount_usd
funding_round_type,
venture,11.748949


### Checkpoint 3: Country Analysis

In [1]:
#top nine countries which have received the highest total funding for the chosen investment type i.e venture
top9=master_frame[master_frame['funding_round_type']=='venture'].groupby('country_code').sum().sort_values('raised_amount_usd',ascending=False).head(9)

SyntaxError: invalid syntax (<ipython-input-1-0fba7f13b411>, line 2)

In [24]:
print (top9)

              raised_amount_usd
country_code                   
USA                4.225108e+11
CHN                3.983542e+10
GBR                2.024563e+10
IND                1.439186e+10
CAN                9.583332e+09
FRA                7.259537e+09
ISR                6.907515e+09
DEU                6.346960e+09
JPN                3.363677e+09


In [25]:
english_speaking_country=['USA','GBR','IND','CAN']

In [26]:
 top9.iloc[top9.index.isin(english_speaking_country)].head(3).index

Index(['USA', 'GBR', 'IND'], dtype='object', name='country_code')

### Checkpoint 4: Sector Analysis 1

In [27]:
#For category list column impute NA with "Blank"
master_frame.category_list.fillna('Blank',inplace=True)
master_frame['primary_sector']=master_frame.category_list.apply(lambda s:s.split('|')[0])

In [28]:
mapping = pd.read_csv('mapping.csv')

In [29]:
#Checking fro NULL values 
mapping.isnull().sum()

category_list                              1
Automotive & Sports                        0
Blanks                                     0
Cleantech / Semiconductors                 0
Entertainment                              0
Health                                     0
Manufacturing                              0
News, Search and Messaging                 0
Others                                     0
Social, Finance, Analytics, Advertising    0
dtype: int64

In [30]:
#From above we can see category_list where we have one NAN
mapping[mapping['category_list'].isnull()]

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0


In [31]:
#So main sector corresponfing to NAN is Blank so we can replcae with "Blank"
mapping['category_list'].fillna('Blank',inplace=True)

In [32]:
#Also observed in mapping data frame charater na is replace by digit 0 so we need to correct it 
mapping[mapping.category_list.str.contains('[0]')]['category_list']

20              Alter0tive Medicine
22                         A0lytics
59                Big Data A0lytics
79                Business A0lytics
89                          Can0bis
91                 Career Ma0gement
103                   Chi0 Internet
114                 Cloud Ma0gement
145               Contact Ma0gement
198        Digital Rights Ma0gement
199                  Digital Sig0ge
210              Document Ma0gement
223                Educatio0l Games
240                Energy Ma0gement
243                  Enterprise 2.0
256                 Event Ma0gement
273                          Fi0nce
274               Fi0nce Technology
275              Fi0ncial Exchanges
276               Fi0ncial Services
281                 Fleet Ma0gement
310                       Gover0nce
353              Identity Ma0gement
370            Innovation Ma0gement
372    Intellectual Asset Ma0gement
384            Investment Ma0gement
389                    IT Ma0gement
391                       Jo

In [33]:
import re 
mapping['category_list']=mapping['category_list'].apply(lambda s : re.sub('((?<=[a-zA-Z])|^)[0]((?=[a-zA-Z ])|$)', 'na', s))

In [34]:
#Settting index name same as category list 
mapping.index=mapping.category_list

In [35]:
#New column for storing main sector
mapping['main_sector']='Default'

In [36]:
for x in mapping.columns[1:]:
    mapping.loc[mapping[x]==1,'main_sector']=x

In [37]:
mapping[['category_list','main_sector']].head(10)

,category_list,main_sector
category_list,,
Blank,Blank,Blanks
3D,3D,Manufacturing
3D Printing,3D Printing,Manufacturing
3D Technology,3D Technology,Manufacturing
Accounting,Accounting,"Social, Finance, Analytics, Advertising"
Active Lifestyle,Active Lifestyle,Health
Ad Targeting,Ad Targeting,"Social, Finance, Analytics, Advertising"
Advanced Materials,Advanced Materials,Manufacturing
Adventure Travel,Adventure Travel,Automotive & Sports


In [38]:
#merging master_funding and mapping dataframe
master_frame=pd.merge(master_frame,mapping[['category_list','main_sector']],
        left_on='primary_sector',
        right_on='category_list',
        how='left',
        )

In [39]:
master_frame.head(5)

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list_x,status,country_code,state_code,region,city,founded_at,primary_sector,category_list_y,main_sector
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,Media,Media,Entertainment
1,/organization/-qounter,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,Application Platforms,Application Platforms,"News, Search and Messaging"
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,Application Platforms,Application Platforms,"News, Search and Messaging"
3,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,Apps,Apps,"News, Search and Messaging"
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,Curated Web,Curated Web,"News, Search and Messaging"


### Checkpoint 5: Sector Analysis 2

In [42]:
(master_frame['country_code']=='USA') and (master_frame['funding_round_type']='venture')

SyntaxError: invalid syntax (<ipython-input-42-e65346465e93>, line 1)